In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
! pip install torch
! pip install transformers
! pip install langchain_community
! pip install langchain
! pip install langchain-huggingface
! pip install langchain_experimental
! pip install langchain_chroma
! pip install langchainhub
! pip install unstructured


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.

In [4]:
! pip install langchain_community

## Loading Documents 

In [5]:
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from transformers import pipeline
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Load URLs
urls = [
    'https://en.wikipedia.org/wiki/Menstruation', 'https://kidshealth.org/en/teens/menstruation.html',
    'https://www.nhs.uk/conditions/periods/', 'https://my.clevelandclinic.org/health/articles/10132-menstrual-cycle',
    'https://www.thewomens.org.au/health-information/periods/periods-overview/about-periods',
    'https://www.mayoclinic.org/healthy-lifestyle/womens-health/in-depth/menstrual-cycle/art-20047186',
    'https://www.mayoclinic.org/diseases-conditions/menopause/symptoms-causes/syc-20353397',
    'https://en.wikipedia.org/wiki/Menopause', 'https://www.nia.nih.gov/health/menopause/what-menopause',
    'https://en.wikipedia.org/wiki/Ovulation', 
    'https://www.westsuburbanmc.com/the-role-of-hormones-in-the-menstrual-cycle','https://en.wikipedia.org/wiki/Polycystic_ovary_syndrome','https://www.obgynnebraska.com/blog/10-common-womens-health-issues-to-know-about'
]

# Load documents
data_loader = UnstructuredURLLoader(urls=urls)
data = data_loader.load()


## Splitting Documents 

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(data)
print("Total number of documents:", len(docs))

Total number of documents: 377


## Embed and Retrieve the documents 

In [7]:
embedding_model = HuggingFaceEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_model)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":10})

<ipython-input-7-c7667c139f4f>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings()
<ipython-input-7-c7667c139f4f>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Bi-encoder for re-ranking 

In [8]:
bi_encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def bi_encoder_rerank(query, retrieved_docs, top_n=3):
    # Encode query and documents
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    doc_embeddings = bi_encoder.encode([doc.page_content for doc in retrieved_docs], convert_to_tensor=True)

    # Compute similarity scores
    scores = torch.nn.functional.cosine_similarity(query_embedding, doc_embeddings)

    # Sort documents by score
    reranked_docs = sorted(zip(retrieved_docs, scores), key=lambda x: x[1], reverse=True)

    return [doc for doc, _ in reranked_docs[:top_n]]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## RAG Pipeline

In [14]:
import torch
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

model_id = "microsoft/phi-2"

# Load the Microsoft Phi-2 model for text generation
text_generation_pipeline = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    max_new_tokens=200,
    device=0,
    return_full_text=False,
    do_sample=False,
    eos_token_id=50256
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Structured prompt to enforce concise Q&A format
prompt_template = """
You are an expert on menstruation. Based on the following context, answer the question accurately and concisely. Do NOT include extra information. Do NOT generate unrelated texts.


Context:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create LLM chain
llm_chain = prompt | llm | StrOutputParser()

# Function to retrieve documents, rerank, and generate an answer
def retrieve_and_answer(question):
    # Retrieve top documents from Chroma
    retrieved_docs = retriever.invoke(question)
    reranked_docs = bi_encoder_rerank(question, retrieved_docs)
    context = "\n".join([doc.page_content for doc in reranked_docs])

    # Invoke LLM
    response = llm_chain.invoke({"context": context, "question": question})

    return response

context_1 = """
Menstruation is the shedding of the uterine lining when pregnancy does not occur. It is regulated by estrogen and progesterone.
"""
question_1 = "What causes menstruation to occur?"

context_2 = """
Menstrual pain (dysmenorrhea) is caused by prostaglandins triggering uterine contractions. Conditions like endometriosis can worsen it.
"""
question_2 = "Why do some people experience pain during menstruation?"

# Run the chain with Menstruation-related input
response_1 = llm_chain.invoke({"context": context_1, "question": question_1})
response_2 = llm_chain.invoke({"context": context_2, "question": question_2})

# Print Outputs
print("Answer 1:", response_1)
print("Answer 2:", response_2)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer 1: Menstruation occurs when pregnancy does not occur. It is regulated by estrogen and progesterone.

Answer 2: Some people experience pain during menstruation due to prostaglandins triggering uterine contractions. Conditions like endometriosis can worsen it.



## Test it with queries 

In [ ]:
context = "Menstruation is regulated by hormones like estrogen and progesterone, which control the shedding of the uterine lining."
question = "Which are the other mammals that mensturate?"

output = llm_chain.invoke({"context": context, "question": question,"stop": ["\n\n"] })
print(output)

# Calculate Metrics

In [ ]:
def evaluate_ranking_metrics(query, retrieved_docs, relevant_docs, top_k=3):
    reranked_docs = bi_encoder_rerank(query, retrieved_docs, top_n=top_k)

    # Extract content for evaluation
    reranked_texts = [doc.page_content for doc in reranked_docs]

    # Mean Reciprocal Rank (MRR)
    mrr = 0
    for idx, doc in enumerate(reranked_texts):
        if doc in relevant_docs:
            mrr = 1 / (idx + 1)
            break

    # Precision@k
    precision = sum(1 for doc in reranked_texts if doc in relevant_docs) / top_k

    # Recall@k
    recall = sum(1 for doc in reranked_texts if doc in relevant_docs) / len(relevant_docs)

    print(f"MRR: {mrr:.4f}, Precision@{top_k}: {precision:.4f}, Recall@{top_k}: {recall:.4f}")
    return mrr, precision, recall

# BLEU Score Calculation
def evaluate_bleu(generated_answer, reference_answer):
    reference_tokens = [nltk.word_tokenize(reference_answer)]
    generated_tokens = nltk.word_tokenize(generated_answer)

    bleu_score = sentence_bleu(reference_tokens, generated_tokens)

    print(f"BLEU Score: {bleu_score:.4f}")
    return bleu_score

# Example Conversation
answer = retrieve_and_answer("Why do some people experience pain during menstruation?")
evaluate_bleu(answer, "Menstrual cramps are caused by uterine contractions triggered by prostaglandins.")

answer = retrieve_and_answer("How can they reduce the pain?")
evaluate_bleu(answer, "Pain relief can be achieved through medications, heat therapy, and lifestyle changes.")
